In [1]:
!pip install segmentation_models
import tensorflow as tf
import cv2
import numpy as np
import os
import glob
import keras
os.environ["SM_FRAMEWORK"] = "tf.keras"
import segmentation_models as sm
import matplotlib.pyplot as plt

Segmentation Models: using `tf.keras` framework.


In [2]:
SIZE_X = 512
SIZE_Y = 512
n_classes = 7
channels_to_keep = [0,1,2,5]
BATCH_SIZE = 8
AUTOTUNE = tf.data.AUTOTUNE
IMAGENET_MEAN = np.array([0.485, 0.456, 0.406])
IMAGENET_STD = np.array([0.229, 0.224, 0.225])

In [3]:
import albumentations as A
from albumentations.core.composition import OneOf
import cv2

def preprocess_and_augment(image, mask):
#     image = image.numpy()
#     mask = mask.numpy()
#     image, mask = augment_image(image, mask)  # Apply augmentations
#     image = tf.convert_to_tensor(image, dtype=tf.float32)
#     mask = tf.convert_to_tensor(mask, dtype=tf.float32)
    image = image / 255.0
    image = (image - IMAGENET_MEAN) / IMAGENET_STD  # Apply ImageNet normalization
#     print(mask.shape)
    mask = tf.squeeze(mask, axis=-1)
#     print(mask.shape)
    mask = tf.one_hot(tf.cast(mask, tf.int32), depth=n_classes)
#     print(mask.shape)
#     channels_to_keep = [1,2,3,4,5,6]
#     print(n_cla)
    mask = tf.gather(mask, indices=channels_to_keep, axis=-1)

# The shape should now be (512, 512, 4)
#     print(mask.shape)
    return image, mask

/opt/conda/lib/python3.10/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.16 (you have 1.4.15). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [4]:
import tensorflow as tf
import os
from segmentation_models import get_preprocessing

# Configuration

base_path = "/kaggle/input/patches-512-v4/patches"
image_path = os.path.join(base_path, "images")
mask_path = os.path.join(base_path, "masks")

# Loading image datasets
image_dataset = tf.keras.utils.image_dataset_from_directory(
    image_path,
    labels=None,
    color_mode='rgb',
    batch_size=BATCH_SIZE,
    image_size=(SIZE_Y, SIZE_X),
    shuffle=False
)

# Loading mask datasets
mask_dataset = tf.keras.utils.image_dataset_from_directory(
    mask_path,
    labels=None,
    color_mode='grayscale',  # Masks are grayscale
    batch_size=BATCH_SIZE,
    image_size=(SIZE_Y, SIZE_X),
    shuffle=False
)

# Combine datasets
dataset = tf.data.Dataset.zip((image_dataset, mask_dataset))

# Apply preprocessing
dataset = dataset.map(preprocess_and_augment, num_parallel_calls=AUTOTUNE)

# Split into train and validation sets
dataset_size = tf.data.experimental.cardinality(dataset).numpy() * BATCH_SIZE
train_size = int(0.8 * dataset_size)

train_dataset = dataset.take(train_size // BATCH_SIZE)
val_dataset = dataset.skip(train_size // BATCH_SIZE)

# Shuffle and prefetch datasets
train_dataset = train_dataset.shuffle(buffer_size=1000).prefetch(AUTOTUNE)
val_dataset = val_dataset.prefetch(AUTOTUNE)

Found 1786 files.
Found 1786 files.


In [ ]:
for images,masks in dataset.skip(10).take(5):
    plt.figure(figsize=(15, 15))
    # Plot original image
    plt.subplot(3, 3, 1)  # Subplot for the image
    plt.imshow(images[0,:,:,:])  # Show the image in RGB
    plt.title("Original Image")
    plt.axis('off')

    # Loop through each mask channel and overlay on the image
    for i in range(4):  # Assuming 7 classes/mask channels
        plt.subplot(3, 3, i+2)  # Subplots for each mask
        plt.imshow(images[0,:,:,:])  # Show the original image
        plt.imshow(masks[0,:, :, i], cmap='jet', alpha=0.5)  # Overlay mask i
        plt.title(f'Overlay of Mask {i+1}')
        plt.axis('off')

    plt.show()

In [5]:
mask_dataset = tf.keras.utils.image_dataset_from_directory(
    mask_path,
    labels=None,
    color_mode='grayscale',  # Masks are grayscale
    batch_size=BATCH_SIZE,
    image_size=(SIZE_Y, SIZE_X),
    shuffle=True
)

Found 1786 files.


In [6]:
masks = []
for mask_batch in mask_dataset:
    masks.append(mask_batch.numpy())

In [7]:
masks = np.concatenate(masks, axis=0)

In [8]:
import numpy as np
import tensorflow as tf
import segmentation_models as sm

# n_classes = 7  # Number of classes in your segmentation task

def calculate_class_frequencies(masks, n_classes):
    # Initialize an array to store pixel counts for each class
    class_frequencies = np.zeros(n_classes)
    
    # Loop over the masks and count pixels in each class
    for mask in masks:
        # Flatten mask and count occurrences of each class
        unique, counts = np.unique(mask, return_counts=True)
        for cls, count in zip(unique.astype(int), counts):  # Convert cls to int
            if cls < n_classes:  # Ensure cls is within the number of classes
                class_frequencies[cls] += count

    return class_frequencies

def compute_class_weights(frequencies):
    # Inverse frequency to give more weight to less frequent classes
    total_pixels = np.sum(frequencies)
    class_weights = total_pixels / (frequencies + 1e-6)  # Add a small epsilon to avoid division by zero
    
    # Normalize the weights so they sum to 1 (optional)
    class_weights = class_weights / np.sum(class_weights)
    
    return class_weights

def weighted_dice_loss(weights):
    def dice_loss(y_true, y_pred):
        smooth = 1e-6
        y_true_f = tf.keras.backend.flatten(y_true)
        y_pred_f = tf.keras.backend.flatten(y_pred)

        # Multiply each class by its corresponding weight
        weighted_true = y_true_f * weights
        weighted_pred = y_pred_f * weights

        intersection = tf.reduce_sum(weighted_true * weighted_pred)
        return 1 - (2. * intersection + smooth) / (tf.reduce_sum(weighted_true) + tf.reduce_sum(weighted_pred) + smooth)

    return dice_loss

# Calculate class frequencies using the masks variable
class_frequencies = calculate_class_frequencies(masks, len(channels_to_keep))

# Compute class weights based on frequencies
class_weights = compute_class_weights(class_frequencies)
print(class_weights)

# Create a weighted Dice Loss using the calculated weights
# loss = weighted_dice_loss(class_weights)


[0.09994675 0.07499922 0.07448051 0.75057352]


In [9]:
import segmentation_models as sm
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout
# from tensorflow.keras.models import Model

# Set the parameters
BACKBONE = 'efficientnetb7'  
LR = 0.0004  # Learning rate

# Define the model
model = sm.Unet(
    backbone_name=BACKBONE,
    encoder_weights='imagenet',
    input_shape=(SIZE_Y, SIZE_X, 3),  # Input image shape
    classes=len(channels_to_keep),
    activation='softmax'  # For multi-class segmentation
)

# Compile the model with relevant losses and metrics
loss = sm.losses.CategoricalFocalLoss() + sm.losses.DiceLoss(class_weights)  # Combination of losses
metrics = ['accuracy',sm.metrics.IOUScore(threshold=0.5), sm.metrics.FScore(threshold=0.5)]  # IOU and F1

# model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0004), 
#               loss=loss, 
#               metrics=['accuracy'])

# # Model summary
model.summary()


258434480/258434480 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 512, 512,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 256, 256,  │      1,728 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 256, 256,  │        256 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 256, 256,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 256, 256,  │        576 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 256, 256,  │        256 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 256, 256,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 64)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 64)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 16)  │      1,040 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 64)  │      1,088 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 256, 256,  │          0 │ block1a_activati… │
│ (Multiply)          │ 64)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 256, 256,  │      2,048 │ block1a_se_excit… │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_bn  │ (None, 256, 256,  │        128 │ block1a_project_… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_dwconv      │ (None, 256, 256,  │        288 │ block1a_project_… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_bn          │ (None, 256, 256,  │        128 │ block1b_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1b_activation  │ (None, 256, 256,  │          0 │ block1b_bn[0][0]

 Total params: 75,048,532 (286.29 MB)

 Trainable params: 74,735,828 (285.09 MB)

 Non-trainable params: 312,704 (1.19 MB)

In [10]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint(
    'best_model_effib7.keras',  # Path to save the model
    monitor='val_accuracy',  # Metric to monitor ('val_accuracy' if using validation set)
    save_best_only=True,  # Only save the model if the monitored metric improves
    mode='max',  # Save the model with the highest accuracy
    verbose=1
)

In [ ]:
# Phase 1: Train decoder with frozen encoder
for layer in model.layers:
    if 'decoder' not in layer.name.split('_'):
        layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.003), loss=loss, metrics=metrics)

history = model.fit(train_dataset, epochs=15, validation_data=val_dataset,callbacks=[checkpoint])

# Phase 2: Unfreeze encoder and train entire model
for layer in model.layers:
    layer.trainable = True

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=4e-4), loss=loss, metrics=metrics)

history = model.fit(train_dataset, epochs=50, validation_data=val_dataset,callbacks=[checkpoint])


Epoch 1/15


I0000 00:00:1727329867.611349     921 service.cc:145] XLA service 0x7f547c013500 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1727329867.611401     921 service.cc:153]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1727329867.611407     921 service.cc:153]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
2024-09-26 05:52:26.294518: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng0{} for conv (f32[8,32,256,256]{3,2,1,0}, u8[0]{0}) custom-call(f32[8,256,256,256]{3,2,1,0}, f32[32,256,3,3]{3,2,1,0}), window={size=3x3 pad=1_1x1_1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convForward", backend_config={"operation_queue_id":"0","wait_on_operation_queues":[],"cudnn_conv_backend_config":{"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0}} is taking a while...
2024-09-26 05:52:27.484877: E external/local_xla/xla/serv

179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 648ms/step - accuracy: 0.4173 - f1-score: 0.3038 - iou_score: 0.2023 - loss: 0.7317
Epoch 1: val_accuracy improved from -inf to 0.53204, saving model to best_model_effib7.keras
179/179 ━━━━━━━━━━━━━━━━━━━━ 406s 1s/step - accuracy: 0.4176 - f1-score: 0.3042 - iou_score: 0.2026 - loss: 0.7315 - val_accuracy: 0.5320 - val_f1-score: 0.4480 - val_iou_score: 0.3294 - val_loss: 0.6534
Epoch 2/15
179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 649ms/step - accuracy: 0.4990 - f1-score: 0.4286 - iou_score: 0.3030 - loss: 0.6524
Epoch 2: val_accuracy did not improve from 0.53204
179/179 ━━━━━━━━━━━━━━━━━━━━ 183s 890ms/step - accuracy: 0.4990 - f1-score: 0.4286 - iou_score: 0.3030 - loss: 0.6524 - val_accuracy: 0.5045 - val_f1-score: 0.4274 - val_iou_score: 0.3059 - val_loss: 0.6413
Epoch 3/15
179/179 ━━━━━━━━━━━━━━━━━━━━ 0s 651ms/step - accuracy: 0.5135 - f1-score: 0.4506 - iou_score: 0.3235 - loss: 0.6282
Epoch 3: val_accuracy improved from 0.53204 to 0.55175, saving model to bes

In [ ]:
loss, accuracy, iou_score, f1_score = model.evaluate(val_dataset)

print(f"Final Loss: {loss}")
print(f"Final Accuracy: {accuracy}")
print(f"Final IOU Score: {iou_score}")
print(f"Final F1 Score: {f1_score}")